# SAMPIC Waveform Visualization

This notebook demonstrates how to read and visualize SAMPIC waveforms from the unpacked ROOT files.

In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Set up nice plotting defaults
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

## Load the ROOT file

In [ ]:
# Open the ROOT file
file = uproot.open('../output.root')
tree = file['events']

print(f"Number of events in file: {tree.num_entries}")
print(f"\nAvailable branches:")
for branch in tree.keys():
    print(f"  - {branch}")

## Explore the data structure

In [ ]:
# Load the first few events
events = tree.arrays(['sampic_event.hits.channel',
                      'sampic_event.hits.corrected_waveform',
                      'sampic_event.hits.amplitude',
                      'sampic_event.hits.baseline',
                      'sampic_event.hits.peak',
                      'sampic_event.hits.time_instant'], 
                     library='ak', entry_stop=10)

print(f"Loaded {len(events)} events")
print(f"\nFirst event structure:")
print(f"  Number of hits: {len(events[0].sampic_event.hits)}")
if len(events[0].sampic_event.hits) > 0:
    hit = events[0].sampic_event.hits[0]
    print(f"  First hit - Channel: {hit.channel}")
    print(f"  First hit - Waveform length: {len(hit.corrected_waveform)}")
    print(f"  First hit - Amplitude: {hit.amplitude:.2f}")
    print(f"  First hit - Baseline: {hit.baseline:.2f}")

## Plot individual waveforms

In [ ]:
def plot_waveform(event_idx, hit_idx, events):
    """Plot a single waveform from a specific event and hit."""
    hit = events[event_idx].sampic_event.hits[hit_idx]
    waveform = np.array(hit.corrected_waveform)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot waveform
    sample_indices = np.arange(len(waveform))
    ax.plot(sample_indices, waveform, 'b-', linewidth=1.5, label='Corrected waveform')
    
    # Mark baseline
    ax.axhline(y=hit.baseline, color='g', linestyle='--', linewidth=1, label=f'Baseline: {hit.baseline:.2f}')
    
    # Mark peak
    ax.axhline(y=hit.peak, color='r', linestyle='--', linewidth=1, label=f'Peak: {hit.peak:.2f}')
    
    ax.set_xlabel('Sample Index')
    ax.set_ylabel('Amplitude [ADC]')
    ax.set_title(f'Event {event_idx}, Hit {hit_idx} - Channel {hit.channel}\n'
                 f'Amplitude: {hit.amplitude:.2f}, Time: {hit.time_instant:.2f} ns')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Plot first waveform from first event
if len(events[0].sampic_event.hits) > 0:
    plot_waveform(0, 0, events)
    plt.show()

## Plot multiple waveforms from one event

In [ ]:
def plot_event_waveforms(event_idx, events, max_hits=4):
    """Plot multiple waveforms from a single event."""
    event = events[event_idx]
    num_hits = min(len(event.sampic_event.hits), max_hits)
    
    if num_hits == 0:
        print(f"Event {event_idx} has no hits")
        return None
    
    fig, axes = plt.subplots(num_hits, 1, figsize=(12, 4*num_hits), sharex=True)
    if num_hits == 1:
        axes = [axes]
    
    for i, ax in enumerate(axes):
        hit = event.sampic_event.hits[i]
        waveform = np.array(hit.corrected_waveform)
        sample_indices = np.arange(len(waveform))
        
        ax.plot(sample_indices, waveform, 'b-', linewidth=1.5)
        ax.axhline(y=hit.baseline, color='g', linestyle='--', linewidth=1, alpha=0.5)
        ax.axhline(y=hit.peak, color='r', linestyle='--', linewidth=1, alpha=0.5)
        
        ax.set_ylabel('Amplitude [ADC]')
        ax.set_title(f'Hit {i} - Ch {hit.channel} | Amp: {hit.amplitude:.1f} | Time: {hit.time_instant:.2f} ns')
        ax.grid(True, alpha=0.3)
    
    axes[-1].set_xlabel('Sample Index')
    fig.suptitle(f'Event {event_idx} - All Hits', fontsize=14, fontweight='bold', y=1.001)
    plt.tight_layout()
    
    return fig

# Plot first event with multiple hits
plot_event_waveforms(0, events, max_hits=4)
plt.show()

## Analyze waveform properties across many events

In [ ]:
# Load more events for statistics
events_large = tree.arrays(['sampic_event.hits.channel',
                             'sampic_event.hits.amplitude',
                             'sampic_event.hits.baseline',
                             'sampic_event.hits.peak',
                             'sampic_event.hits.time_instant',
                             'sampic_event.hits.tot_value'], 
                            library='ak', entry_stop=1000)

# Flatten to get all hits
import awkward as ak

amplitudes = ak.flatten(events_large.sampic_event.hits.amplitude)
baselines = ak.flatten(events_large.sampic_event.hits.baseline)
peaks = ak.flatten(events_large.sampic_event.hits.peak)
channels = ak.flatten(events_large.sampic_event.hits.channel)
tot_values = ak.flatten(events_large.sampic_event.hits.tot_value)

print(f"Total hits analyzed: {len(amplitudes)}")
print(f"\nAmplitude statistics:")
print(f"  Mean: {np.mean(amplitudes):.2f}")
print(f"  Std:  {np.std(amplitudes):.2f}")
print(f"  Min:  {np.min(amplitudes):.2f}")
print(f"  Max:  {np.max(amplitudes):.2f}")

## Plot distributions

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Amplitude distribution
axes[0, 0].hist(amplitudes, bins=100, alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('Amplitude [ADC]')
axes[0, 0].set_ylabel('Counts')
axes[0, 0].set_title('Amplitude Distribution')
axes[0, 0].set_yscale('log')

# Baseline distribution
axes[0, 1].hist(baselines, bins=100, alpha=0.7, edgecolor='black', color='green')
axes[0, 1].set_xlabel('Baseline [ADC]')
axes[0, 1].set_ylabel('Counts')
axes[0, 1].set_title('Baseline Distribution')

# Channel occupancy
axes[1, 0].hist(channels, bins=64, range=(0, 64), alpha=0.7, edgecolor='black', color='orange')
axes[1, 0].set_xlabel('Channel')
axes[1, 0].set_ylabel('Hits')
axes[1, 0].set_title('Channel Occupancy')

# TOT distribution
axes[1, 1].hist(tot_values, bins=100, alpha=0.7, edgecolor='black', color='purple')
axes[1, 1].set_xlabel('Time over Threshold [ns]')
axes[1, 1].set_ylabel('Counts')
axes[1, 1].set_title('TOT Distribution')
axes[1, 1].set_yscale('log')

plt.tight_layout()
plt.show()

## Peak vs Amplitude correlation

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# Create 2D histogram
h = ax.hist2d(np.array(amplitudes), np.array(peaks), bins=100, cmap='viridis', 
              cmin=1, norm=plt.matplotlib.colors.LogNorm())
plt.colorbar(h[3], ax=ax, label='Counts')

ax.set_xlabel('Amplitude [ADC]')
ax.set_ylabel('Peak [ADC]')
ax.set_title('Peak vs Amplitude Correlation')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary Statistics by Channel

In [ ]:
# Create DataFrame for easier analysis
df = pd.DataFrame({
    'channel': np.array(channels),
    'amplitude': np.array(amplitudes),
    'baseline': np.array(baselines),
    'peak': np.array(peaks),
    'tot': np.array(tot_values)
})

# Group by channel and compute statistics
channel_stats = df.groupby('channel').agg({
    'amplitude': ['count', 'mean', 'std'],
    'baseline': 'mean',
    'peak': 'mean'
}).round(2)

print("\nStatistics by Channel (first 10 channels):")
print(channel_stats.head(10))

## Average Waveform Shape

In [ ]:
# Load waveforms from first 100 events
events_wf = tree.arrays(['sampic_event.hits.corrected_waveform',
                          'sampic_event.hits.baseline'], 
                         library='ak', entry_stop=100)

# Collect all waveforms with non-zero length
all_waveforms = []
for event in events_wf:
    for hit in event.sampic_event.hits:
        wf = np.array(hit.corrected_waveform)
        baseline = hit.baseline
        if len(wf) > 0:
            # Subtract baseline
            all_waveforms.append(wf - baseline)

if len(all_waveforms) > 0:
    # Find maximum length
    max_len = max(len(wf) for wf in all_waveforms)
    
    # Pad shorter waveforms with zeros
    padded_waveforms = np.array([np.pad(wf, (0, max_len - len(wf)), constant_values=0) 
                                  for wf in all_waveforms])
    
    # Compute average and std
    avg_waveform = np.mean(padded_waveforms, axis=0)
    std_waveform = np.std(padded_waveforms, axis=0)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    sample_indices = np.arange(len(avg_waveform))
    ax.plot(sample_indices, avg_waveform, 'b-', linewidth=2, label='Average')
    ax.fill_between(sample_indices, 
                     avg_waveform - std_waveform, 
                     avg_waveform + std_waveform,
                     alpha=0.3, label='±1 σ')
    
    ax.set_xlabel('Sample Index')
    ax.set_ylabel('Amplitude (baseline subtracted) [ADC]')
    ax.set_title(f'Average Waveform Shape ({len(all_waveforms)} hits)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Analyzed {len(all_waveforms)} waveforms")
    print(f"Maximum waveform length: {max_len} samples")
else:
    print("No waveforms found in the data")